In [5]:
import os
folders = [f for f in os.listdir('.') if os.path.isdir(f)]

print(folders)


['.ipynb_checkpoints']


In [13]:
import dxpy

# Get the current project ID (should be auto-attached to Jupyter)
project = dxpy.DXProject(dxpy.WORKSPACE_ID)
print("Attached Project ID:", project.get_id())

project_id = dxpy.WORKSPACE_ID
print(project_id)

data_folder = "/data"


Attached Project ID: project-GyGQppQ09KP7GzZV90fyBggj
project-GyGQppQ09KP7GzZV90fyBggj


In [16]:
folders = dxpy.api.project_list_folder(project_id, {"folder": "/"})["folders"]
print("Folders in project root:", folders)

Folders in project root: ['/20250128_haunting_grackle', '/failed_run', '/files_scripts_processing', '/rna_outputs', '/wgs_outputs']


In [18]:
folder_path = "/wgs_outputs/A1_P1"
files = list(dxpy.find_data_objects(project=project_id, folder=folder_path, return_handler=True))
for file in files:
    print("File:", file.name, "| ID:", file.get_id())

File: A1_P1_markdup_tnscope.vcf.maf.gz | ID: file-GyYzP8j0pyg3GQXqYFyG2jQq
File: A1_P1_markdup_tnscope.vcf.gz.tbi | ID: file-GyYyxxj0q1Jyx8BKBKZqfF1y
File: A1_P1_markdup_tnscope.vcf.gz | ID: file-GyYyxxj0q1JgP2J81BYFj3xk
File: A1_P1_tumor_markdup.bam.bai | ID: file-GyYp73j0q1JVYPfvPvk8yv8j
File: A1_P1_tumor_markdup.bam | ID: file-GyYp73j0q1JgP8QKGVg71FG9
File: A1_P1_markdup.cns.reannotated.tsv | ID: file-GyYyvK008XBK9kXBjBb1Yb4p
File: A1_P1_markdup.bam.bai | ID: file-GyYvz6008ZXX59108KJ9v3vB
File: A1_P1_markdup.bam | ID: file-GyYvz6008ZXqqby4GgV11kX3


In [26]:
file_name = "A1_P1_markdup_tnscope.vcf.gz"

file_obj = next((f for f in files if f.name == file_name), None)

print(file_obj.id)
print(file_obj)

file-GyYyxxj0q1JgP2J81BYFj3xk
<dxpy.DXFile: A1_P1_markdup_tnscope.vcf.gz (file-GyYyxxj0q1JgP2J81BYFj3xk (project-GyGQppQ09KP7GzZV90fyBggj))>


In [28]:
dxpy.download_dxfile(file_obj.id, file_name)

In [31]:
!wget https://ftp.ensembl.org/pub/release-106/variation/vcf/mus_musculus/mus_musculus.vcf.gz

--2025-03-04 20:06:51--  https://ftp.ensembl.org/pub/release-106/variation/vcf/mus_musculus/mus_musculus.vcf.gz
Resolving ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.169
Connecting to ftp.ensembl.org (ftp.ensembl.org)|193.62.193.169|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 907911627 (866M) [application/x-gzip]
Saving to: ‘mus_musculus.vcf.gz’

mus_musculus.vcf.gz 100%[===================>] 865.85M  77.3MB/s    in 10s     

2025-03-04 20:07:02 (86.3 MB/s) - ‘mus_musculus.vcf.gz’ saved [907911627/907911627]



In [32]:
!pip install pysam

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.0/26.0 MB 173.4 MB/s eta 0:00:00


In [33]:
import pysam
import pandas as pd

In [34]:
print(os.listdir())

['.ipynb_checkpoints', 'Untitled1.ipynb', 'test.ipynb', 'A1_P1_markdup_tnscope.vcf.gz', 'mus_musculus.vcf.gz']


In [35]:
vcf = pysam.VariantFile("A1_P1_markdup_tnscope.vcf.gz")

In [38]:
def read_vcf_snvs(vcf_path):
    """
    Reads a VCF file and extracts only SNVs (Single Nucleotide Variants).
    Returns a DataFrame with CHROM, POS, REF, ALT.
    """
    vcf = pysam.VariantFile(vcf_path)  # Open VCF
    snv_list = []

    for record in vcf:
        # Ensure both REF and ALT are single nucleotides (SNV)
        if len(record.ref) == 1 and all(len(alt) == 1 for alt in record.alts):
            for alt in record.alts:
                snv_list.append([record.chrom, record.pos, record.ref, alt])

    return pd.DataFrame(snv_list, columns=["CHROM", "POS", "REF", "ALT"])


def filter_somatic_snvs(sample_vcf, germline_vcf, output_vcf):
    """
    Filters out known germline SNVs from the sample VCF.
    Saves a new VCF file with only somatic SNVs.
    """
    # Read SNVs from sample and germline VCFs
    sample_snvs = read_vcf_snvs(sample_vcf)
    germline_snvs = read_vcf_snvs(germline_vcf)

    # Perform an anti-join (remove known germline SNVs)
    somatic_snvs = sample_snvs.merge(germline_snvs, on=["CHROM", "POS", "REF", "ALT"], how="left", indicator=True)
    somatic_snvs = somatic_snvs[somatic_snvs["_merge"] == "left_only"].drop(columns=["_merge"])

    # Read original sample VCF for metadata retention
    vcf_in = pysam.VariantFile(sample_vcf)
    vcf_out = pysam.VariantFile(output_vcf, "w", header=vcf_in.header)

    # Write only somatic SNVs to the new VCF
    for record in vcf_in:
        if [record.chrom, record.pos, record.ref, record.alts[0]] in somatic_snvs.values.tolist():
            vcf_out.write(record)

    vcf_out.close()
    print(f"Filtered somatic SNVs VCF saved to: {output_vcf}")

In [37]:
sample_vcf_path = "A1_P1_markdup_tnscope.vcf.gz"
germline_vcf_path = "mus_musculus.vcf.gz"
output_vcf_path = "filtered_" + sample_vcf_path
print(output_vcf_path)

filtered_A1_P1_markdup_tnscope.vcf.gz


In [ ]:
filter_somatic_snvs(sample_vcf_path, germline_vcf_path, output_vcf_path)